# 04 - évaluation et LIME
Rapports, matrices de confusion, explications.

In [ ]:
from pathlib import Path
import sys
from subprocess import run
import json
ROOT = Path.cwd().resolve()
for _ in range(4):
    if (ROOT / 'Data').exists():
        break
    ROOT = ROOT.parent
sys.path.append(str(ROOT))
TEST_DIR = ROOT / 'Data/raw/test'   # basculer sur Data/processed/test_224 si besoin
CONF_PATH = ROOT / 'Visualisation/confusion_matrix.png'
REPORT_PATH = ROOT / 'Monitoring/output/metrics.json'
CURVES = ROOT / 'Visualisation/training_curves.png'
PY = sys.executable
CLASSES = ['Chao','Ervas','Milho','Milho_ervas']


In [ ]:
cmd = [    PY, 'Model/training/evaluate.py',    '--data-dir', str(TEST_DIR),    '--weights', str(ROOT/'Model/weights/best_model.pt'),    '--confusion-path', str(CONF_PATH),    '--report-path', str(REPORT_PATH),    '--metrics-json', str(ROOT/'Monitoring/output/metrics.json'),    '--training-curves', str(CURVES)]print(' '.join(cmd))run(cmd, check=True)print('Confusion matrix saved to', CONF_PATH)print('Report saved to', REPORT_PATH)


In [ ]:
# LIME sur quelques images
from lime import lime_image
from skimage.segmentation import mark_boundaries
import torch
from torchvision import datasets
from Model.training.utils import build_transforms, create_model, map_aliases

WEIGHTS = ROOT / 'Model/weights/best_model.pt'
IMAGE_SIZE = 224

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = datasets.ImageFolder(TEST_DIR, transform=build_transforms(IMAGE_SIZE, augment='none'))
class_names = map_aliases(dataset.classes)
model = create_model('resnet18', num_classes=len(class_names), pretrained=False, dropout=0.3)
ckpt = torch.load(WEIGHTS, map_location=device)
model.load_state_dict(ckpt.get('model_state', ckpt), strict=False)
model.eval().to(device)

preprocess = build_transforms(IMAGE_SIZE, augment='none')
explainer = lime_image.LimeImageExplainer()

samples = [list((TEST_DIR/cls).glob('*.jpg'))[0] for cls in CLASSES]

for img_path in samples:
    img = Image.open(img_path).convert('RGB')
    np_img = np.array(img)
    def predict_fn(batch):
        batch_t = torch.stack([preprocess(Image.fromarray(b)).to(device) for b in batch])
        with torch.no_grad():
            logits = model(batch_t)
            probs = torch.softmax(logits, dim=1).cpu().numpy()
        return probs
    explanation = explainer.explain_instance(np_img, predict_fn, top_labels=1, num_samples=1000)
    top_label = explanation.top_labels[0]
    temp, mask = explanation.get_image_and_mask(top_label, positive_only=True, num_features=5, hide_rest=False)
    plt.figure(figsize=(5,3))
    plt.imshow(mark_boundaries(temp, mask))
    plt.title(f"{img_path.parent.name} -> {class_names[top_label]}")
    plt.axis('off')
    plt.show()
